## Import libraries

In [1]:
#!pip install -U deep-translator
#!pip install pymorphy2
#!pip install natasha
#!pip install navec

In [46]:
import pandas as pd
import json
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import os
import json
import gensim
from gensim.utils import simple_preprocess
from gensim.models import word2vec
import gensim.corpora as corpora
from deep_translator import GoogleTranslator
from navec import Navec
from ipymarkup import show_dep_ascii_markup as show_markup
from razdel import sentenize, tokenize
from slovnet import Syntax
from slovnet import Morph
from slovnet import NER
from navec import Navec
import math
import requests
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter, OrderedDict
import pymorphy2
from numpy import dot
from numpy.linalg import norm
from nltk.stem import SnowballStemmer
from ipymarkup import show_span_ascii_markup as show_markup
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Маргарита\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)
morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
morph.navec(navec)
syntax = Syntax.load('slovnet_syntax_news_v1.tar')
syntax.navec(navec)

Syntax(
    infer=SyntaxInfer(
        model=Syntax(
            emb=WordShapeEmbedding(
                word=NavecEmbedding(
                    id='news_v1_1B_250K_300d_100q',
                    indexes=Weight(
                        shape=[250002,
                         100],
                        dtype='uint8',
                        array=array([[176, 222, 248, ..., 244, 183, 191],
                               [215, 200, 168, ..., 120, 217,  21],
                               [ 83, 174,  54, ..., 106,  88, 251],
                               ...,
                               [133, 125, 123, ..., 124,  94,  24],
                               [183,  49, 180, ..., 151, 167,  68],
                               [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)
                    ),
                    codes=Weight(
                        shape=[100,
                         256,
                         3],
                        dtype='float32',
                    

#### Initialize Google Translator

In [4]:
google_traslator = GoogleTranslator(source='auto', target='ru')
def translate_chunck(amount_per_chunck, df, num_of_interation):
    whole_df = pd.DataFrame()
    num_of_sentences = len(df.index)
    if num_of_interation==-1:
        num_of_interation = math.floor(num_of_sentences/amount_per_chunck)
    start=0
    end=0
    for i in range(num_of_interation):
        end = end+amount_per_chunck
        sub_df = df[start:end]
        start = start+amount_per_chunck
        sub_df['Russian_sentence'] = sub_df['Sentence'].apply(google_traslator.translate, src='id', dest='en')
        whole_df = pd.concat([whole_df, sub_df])
    if num_of_interation==-1:
        sub_df = df[start:]
        sub_df['Russian_sentence'] = sub_df['Sentence'].apply(google_traslator.translate, src='id', dest='en')
        whole_df = pd.concat([whole_df, sub_df])
    return whole_df

#### Initialize Lemmatizer

In [43]:
stop_words = ['и', 'в', 'во','не', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все','она', 'так', 'его', 'но', 'да', 'ты' 'к', 'у', 'же', 'вы','за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из']
#stopwords.words('russian')
#a lot of "stop words" could be usefull
#Lemmatize with MorphAnalyzer
#Bag of words creation
#If we want to extend list of stop words
newStopWords = []
stop_words.extend(newStopWords)
#make a bag od words
def sent_to_words(sentence):
    # deacc=True removes punctuations
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [remove_stopwords_from_sent(doc) for doc in texts]
def remove_stopwords_from_sent(sent):
    return [word for word in simple_preprocess(str(sent)) if word not in stop_words]
morph_pymorphy = pymorphy2.MorphAnalyzer()
def get_array_of_words(sentence):
    data_words = list(sent_to_words(sentence))
    # remove stop words
    data_words = remove_stopwords(data_words)
    return data_words[0]
def lemmatize_count(words):
    a = Counter()
    for i in words:
        p = morph_pymorphy.parse(i)[0]
        word = p.normal_form
        a[word]+=1
    return a
def amount_of_words(sentence):
    words = get_array_of_words(sentence)
    return len(words)
def create_bag_of_words(sentence):
    words = get_array_of_words(sentence)
    return lemmatize_count(words)
def lemmatize_sentence(sentence):
    array_of_words=[]
    words = get_array_of_words(sentence)
    for i in words:
        p = morph_pymorphy.parse(i)[0]
        word = p.normal_form
        array_of_words.append(word)
        sentence=' '.join(array_of_words)
    return sentence

## Import data

### 1. Cross-Lingual Argumentation Mining for Russian Texts (7th article)

In [6]:
#leave only needed data function
def only_russian_texts(lines):
    sentences={}
    annotations={}
    for line in lines:
        if 'lang="en"' not in line:
            if '<edu' in line:
                entries = re.split("(<|>)", line)
                key = entries[2].replace('edu id="e', '').replace('" lang="', '').replace('en"', '').replace('ru" trans="human"', '')     
                value = entries[4]
                sentences[key] = value
            elif '<adu' in line:
                entries = re.split("\s", line)
                #entries[3] = entries[3].replace('id="a', '')
                key = entries[3].replace('id="a', '').replace('"', '')
                value = entries[4].replace('type="', '').replace('"/>', '')
                annotations[key] = value
    return sentences, annotations
#create dataframe function
def create_dataframe(sentences, annotations):
    df = pd.DataFrame()
    df['sentence'] = sentences.values()
    arguments = []
    for key in sentences.keys():
        if key in annotations:
            arguments.append(annotations[key])
        else:
            arguments.append('none')
    df['argument'] = arguments
    return df

In [7]:
#import
folders = ["Data1. translated_part1_human", "Data2. translated_part2_human"]
documents = []
for folder_name in folders:
    for filename in os.listdir(folder_name):
        try:
            with open(os.path.join(folder_name, filename), 'r', encoding="utf8") as f:
                text = f.readlines()
                documents.append(text)
        except:
            with open(os.path.join(folder_name, filename), 'r') as f:
                text = f.readlines()
                documents.append(text)

In [8]:
#put all data in one dataframe
pro_con_1 = pd.DataFrame()
for document in documents:
    sentences, annotations = only_russian_texts(document)
    new_df = create_dataframe(sentences, annotations)
    pro_con_1 = pd.concat([pro_con_1, new_df])

In [9]:
pro_con_1

,sentence,argument
0,"Да, очень раздражает и утомляет все время сорт...",opp
1,На кухне воняют три разных мешка для мусора. И...,opp
2,Но все же Германия производит слишком много му...,pro
3,"и теряется слишком много ресурсов, когда сжига...",pro
4,"Мы, берлинцы, должны воспользоваться шансом и ...",pro
...,...,...
0,Многодетные семьи не обязательно считаются луч...,pro
1,Наличие большой семьи заставляет каждого члена...,pro
2,"Можно утверждать, что в большой семье у детей ...",opp
3,Однако это также может вызвать беспокойство и ...,pro


In [10]:
#pro_con_1.to_csv(r'C:\Users\Маргарита\Desktop\Passau Uni\Магистерская диссертация\PRO_CON_1.csv', sep=',')

### 2. Canary (essays)

In [11]:
def add_numberodessay(message, index):
    return str(index)+message
def split_column(message):
    entries = re.split("\s", message)
    return entries
def type_column(splitted):
    if splitted[0] == 'MajorClaim':
        return 'MajorClaim'
    elif splitted[0] == 'Claim':
        return 'Claim'
    elif splitted[0] == 'Premise':
        return 'Premise'
    else:
        return ''
def return_splitted_word(splitted, index):
    message = splitted[index]
    if 'Arg1:' in message:
        message=message.replace('Arg1:','')
    elif 'Arg2:' in message:
        message=message.replace('Arg2:','')
    elif 'Stance' in message:
        message = splitted[2]
    return message
def create_relation(number, df_essays):
    if number.find('T'):
        try:
            element = df_essays[df_essays['relation']==number]
            value = element['word_1'].to_numpy()[0]
            if element[0].to_numpy()[0].find('A'):
                name = element['word_2'].to_numpy()[0]
            else:  
                name = element['word_3'].to_numpy()[0]
            return str(value)+' '+str(name)
        except:
            return 'None None'
    else:
        return 'None None'

In [12]:
#import
df_essays = pd.DataFrame()
count = 0 
for filename in os.listdir("Data3. essays"):
    count+=1
    path = 'Data3. essays/'+filename
    try:
        new_df = pd.read_csv(path, sep='\t', engine='python', header=None)
        new_df[0] = new_df.apply(lambda row: add_numberodessay(row[0], count), axis=1)
        new_df['essay_num'] = count
        df_essays = pd.concat([df_essays, new_df])
    except:
        print(filename)

essay120.ann
essay163.ann
essay172.ann
essay174.ann
essay182.ann
essay202.ann
essay265.ann
essay277.ann
essay318.ann
essay393.ann


In [13]:
#import full texts of essays
folder_name = "Data3. full_essays"
full_essays = {}
count = 0 
for filename in os.listdir(folder_name):
    count+=1
    with open(os.path.join(folder_name, filename), 'r', encoding="utf8") as f:
        text = f.read()
        full_essays[count] = text

In [14]:
pd_full_essays = pd.DataFrame()
pd_full_essays['essay_num'] = full_essays.keys()
pd_full_essays['full_text'] = full_essays.values()

In [15]:
def get_topic(text):
    array = text.split('\n')
    return array[0]

In [16]:
def get_text_without_topic(text):
    array = text.split('\n')
    only_text_array = array[1:]
    only_text=' '.join(only_text_array)
    return only_text

In [17]:
pd_full_essays['Topic'] = pd_full_essays.apply(lambda row: get_topic(row['full_text']), axis=1)

In [18]:
pd_full_essays['Text'] = pd_full_essays.apply(lambda row: get_text_without_topic(row['full_text']), axis=1)

In [19]:
pd_full_essays.head()

,essay_num,full_text,Topic,Text
0,1,Should students be taught to compete or to coo...,Should students be taught to compete or to coo...,It is always said that competition can effect...
1,2,More people are migrating to other countries t...,More people are migrating to other countries t...,The last 50 years have seen an increasing num...
2,3,International tourism is now more common than ...,International tourism is now more common than ...,The last decade has seen an increasing number...
3,4,International tourism is now more common than ...,International tourism is now more common than ...,The last 50 years have seen a significant inc...
4,5,Living and studying overseas\n\nIt is every st...,Living and studying overseas,It is every student's desire to study at a go...


In [20]:
df_essays.head()

,0,1,2,essay_num
0,1T1,MajorClaim 503 575,we should attach more importance to cooperatio...,1
1,1T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,1
2,1T3,Claim 591 714,"through cooperation, children can learn about ...",1
3,1A1,Stance T3 For,None,1
4,1T4,Premise 716 851,What we acquired from team work is not only ho...,1


In [21]:
def get_sentences_from_texts_df(essay_num, text):
    df = pd.DataFrame()
    sentences = re.split(r"\.\s+|\!|\?", text)
    df['Sentence'] = sentences
    df['essay_num'] = essay_num
    return df

In [22]:
all_sentences_essays = pd.DataFrame()
for index in range(len(pd_full_essays.index)):
    new_df = get_sentences_from_texts_df(pd_full_essays['essay_num'][index], pd_full_essays['Text'][index])
    all_sentences_essays = pd.concat([all_sentences_essays, new_df])

In [23]:
#We split the first column in which type, start and end are kept
df_essays['splitted'] = df_essays.apply(lambda row: split_column(row[1]), axis=1)
df_essays['type_ADU'] = df_essays.apply(lambda row: type_column(row['splitted']), axis=1)
df_essays['word_1'] = df_essays.apply(lambda row: return_splitted_word(row['splitted'], 0), axis=1)
df_essays['word_2'] = df_essays.apply(lambda row: return_splitted_word(row['splitted'], 1), axis=1)
df_essays['word_3'] = df_essays.apply(lambda row: return_splitted_word(row['splitted'], 2), axis=1)
#drop unneeded columns
df_essays = df_essays.drop(columns = 1)
df_essays = df_essays.drop(columns = ['splitted'])

In [24]:
#defined relations
df_essays['essay_num'] = df_essays['essay_num'].apply(str)
df_essays['relation'] = df_essays['essay_num']+df_essays['word_2']
df_essays['relation_type'] = df_essays.apply(lambda row: create_relation(row[0], df_essays), axis=1)
df_essays['relation_type_'] = df_essays['relation_type'].apply(lambda x: x.split()[0])
df_essays['relation_to'] = df_essays['relation_type'].apply(lambda x: x.split()[1])

In [25]:
#drop unneeded columns
df_essays = df_essays.drop(columns = ['relation_type','word_1', 'word_2','word_3', 'relation'])

In [26]:
#Leave only sentences
df_essays = df_essays[df_essays[0].str.contains('T')]

In [27]:
df_essays['essay_num'] = df_essays['essay_num'].apply(lambda x: int(x))

In [28]:
essays_final = pd.merge(df_essays,pd_full_essays,on='essay_num',how='inner')

In [29]:
# test_402 = essays_final[essays_final['essay_num'] == 402]['Expression']
# for i in test_402:
#     print(i,'\n')

In [30]:
essays_final.columns = ['Expression_Id', 'Expression', 'essay_num', 'type_ADU', 'relation_type_','relation_to', 'full_text','Topic', 'Text']

In [31]:
def get_full_sentence(expression, text):
    sentences = re.split(r"\.\s+|\!|\?", text)
    for sentence in sentences:
        if sentence.find(expression)>-1:
            return sentence
    return 'Not found'

In [32]:
essays_final['Sentence'] = essays_final.apply(lambda row: get_full_sentence(row['Expression'], row['full_text']), axis=1)

In [33]:
print(len(essays_final['Sentence'].index))
print(len(essays_final['Sentence'].unique()))

5920
5313


In [34]:
ADUs_essays = essays_final['Sentence'].unique()

In [35]:
all_sentences_essays['ADU'] = all_sentences_essays['Sentence'].apply(lambda x: True if x in ADUs_essays else False)

In [36]:
#test_402_false = all_sentences_essays[(all_sentences_essays['ADU']==False) & (all_sentences_essays['essay_num']==402)]

#### Translate. It takes too much time, so I exported translated date to csv

In [37]:
# amount_per_chunck=1000
# df = all_sentences_essays
# whole_df = pd.DataFrame()
# num_of_sentences = len(df.index)
# num_of_interation = math.floor(num_of_sentences/amount_per_chunck)
# start=0
# end=0
# for i in range(num_of_interation):
#     end = end+amount_per_chunck
#     sub_df = df[start:end]
#     start = start+amount_per_chunck
#     sub_df['Russian_sentence'] = sub_df['Sentence'].apply(google_traslator.translate, src='id', dest='en')
#     whole_df = pd.concat([whole_df, sub_df])
# sub_df = df[start:]
# sub_df['Russian_sentence'] = sub_df['Sentence'].apply(google_traslator.translate, src='id', dest='en')
# whole_df = pd.concat([whole_df, sub_df])

In [38]:
# whole_df.to_csv(r'C:\Users\Маргарита\Desktop\Passau Uni\Магистерская диссертация\DF_ESSAYS.csv', sep=',')

In [39]:
#for training argumentative and non-argumentative
essays_russian_ADUs = pd.read_csv(r'C:\Users\Маргарита\Desktop\Passau Uni\Магистерская диссертация\DF_ESSAYS.csv')

In [40]:
essays_russian = essays_russian_ADUs.copy()
del essays_russian['essay_num']
del essays_russian['ADU']
del essays_russian['Unnamed: 0']

#### Get features from Russian sentences

In [41]:
essays_russian_features = essays_russian.copy()
essays_russian_features.head()

,Sentence,Russian_sentence
0,It is always said that competition can effect...,"Всегда говорят, что конкуренция может эффектив..."
1,"In order to survive in the competition, compan...","Чтобы выжить в конкурентной борьбе, компании п..."
2,"However, when we discuss the issue of competit...",Однако когда мы обсуждаем вопрос о конкуренции...
3,"From this point of view, I firmly believe that...","С этой точки зрения я твердо верю, что мы долж..."
4,"First of all, through cooperation, children ca...","Во-первых, благодаря сотрудничеству дети могут..."


In [44]:
essays_russian_features['amount_of_words'] = essays_russian_features.apply(lambda row: amount_of_words(row['Russian_sentence']), axis=1)
essays_russian_features['bag_of_words'] = essays_russian_features.apply(lambda row: create_bag_of_words(row['Russian_sentence']), axis=1)
essays_russian_features['lemmatized_sentence'] = essays_russian_features.apply(lambda row: lemmatize_sentence(row['Russian_sentence']), axis=1)

#### Add unigrams

In [47]:
corpus_essays = essays_russian_features['lemmatized_sentence'].array
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus_essays)
unigrams_essays = vectorizer.get_feature_names_out()

In [52]:
for sent in essays_russian_features['lemmatized_sentence'].array:
    print(sent)

всегда говорить что конкуренция мочь эффективно способствовать развитие экономика
чтобы выжить конкурентноить борьба компания продолжать свой продукт услуга результат процветать общество
однако когда мы обсуждать вопрос конкуренция или сотрудничество мы интересовать общество целое развитие всей жизнь отдельный человек
этой точка зрение твёрдо верить что мы должный придавать больший значение сотрудничество начальноить школа
первый благодаря сотрудничество ребёнок мочь узнать навык межличностный общение который важный для будущей жизнь весь учиться
что мы приобрести благодаря командноить работа это достигнуть той цель другой что более важно ладить другой
процесс сотрудничество ребёнок мочь научиться тот прислушиваться мнение другой общаться другой мыслить комплексно даже идти компромисс другой член команда когда возникать конфликт
этот навык помогать они ладить другой человек пригодиться они весь жизнь
другой сторона значение конкуренция заключаться тот стать более совершенный чтобы одер

однако доступ интернет один величаинуть нововведение современный общество человечество мочь освоить традиционный метод отображать онлаин любой время любой место
кроме тот технология обогащать способ отображение традиционный культурный наследие делать более яркий привлекательный
например китая успешно продвигать традиционный метод всемирноить выставка шанхай использовать различный современный технологический метод такой робот светодиодный экран
сам дело многие турист весь мир восхищаться идеальный синтез современный технология традиционноить культура
другой сторона некоторый человек обеспокоить тем что технология мочь оказать пагубный влияние традиционный образ жизнь
они утверждать что человек особенно молодёжь большой без ум свежий продвинуть вещей такой цифровой продукт поэтому становиться равнодушный традиционный технология
считать что это проблема потому что человек склонный следовать тенденция традиция последний год
поэтому прийти вывод что технология деиствительный способствовать 

моей точка зрение три основной причина это расширение знание подготовка карьера новыя опыт
первый для большинство людея университет являться идеальный место для развитие свой понимание
правда говорить этой замечательноить академической среда каждый предоставляться разнообразие гигантский количество информация множество профессор хорошо подготовить преподавателей
такой образ человек мочь укреплять свой знание
более тот указание учителей достаточный инфраструктура такой лаборатория библиотека студент мочь проводить исследование каждый академический аспект
такой образ они мочь получить более глубокий представление тот что они изучать
второй учёба университет означать что полностью переслать другой страница своей жизнь которой появиться много новый впечатление
количество людея который поступать университет огромный большой чем любой школа где учиться один человек
такой образ студент предоставляться такой фантастическия шанс создать новый широкий отношение больший количество людея любой уго

первый причина заключаться тот что использовать разный источник информация мы мочь гарантировать достоверность
некоторый человек мочь вы намеренно поэтому делать вывод быть уверенный информация довольно неразумно
например россииский новость сообщаться что воин между осетией грузией быть инициировать грузией европеиский новость сообщаться что инициировать россия
через некоторый время ситуация проясниться до это деиствие россия уже осудить многие
второй очень важно знать факт какой событие или проблема оценка потому что это мочь помочь вы сделать собственный вывод или оценка
иногда бывать очень сложно узнать факт особенно интернет поэтому приходиться просматривать разный источник прежде чем удаться наити достоверный факт
возможно принять неправильный решение потому что что использовать чей личный мнение качество источник информация сам дело оно неверный
например несколько год назад москва соль подорожать два раз один неделя потому что человек думать что скоро соль магазин быть
информация

поэтому курение ресторан должный быть строго запретить это делать ресторан загрязнить место угрожать здоровье каждый посетитель
кроме тот хотя некоторый клиент беспокоиться о угроза который представлять для они пассивный курение отвратительный дым серьёзно влиять они аппетит или другой слово портить они еда делать неприятной
дым выдыхаемый курильщик часто вызывать другой людея чувство дискомфорт или отвращение что влиять они настроение время еда
это другой сторона мочь косвенно снизить популярность ресторан потому что маленький людея готовый есть там свой неприятный опыт
наконец курение приносить польза сам курильщик
это ухудшать здоровье делать более подверженный респираторный заболевание наносить ущерб общественный имидж репутация
это особенно верно когда курильщик являться популярный политический деятель или знаменитость потому что большинство людея считать курение ресторан невежливый поведение
этот момент общественность вероятно поверить что курильщик хватать элементарноить вежливо

голосовой электронный почта или разговор лицо лицо иногда заставлять сдержать людея чувствовать себя очень уверенно
например когда сталкиваться родитель мочь высказать каждый мысль выразить последовательно
они мочь остановить сказать что против это заставить забыть что хотеть сказать
любой мочь столкнуться этой проблемои один раз любой разговор
электронный почта помогать идеально оформить мой идея сделать убедительноить
целое сравнение весь другой способ общение деиствительный считать что электронный почта лучшие способ для большинство людея
удобство экономить наш время
кроме тот честность необходимый для каждый разговор беглость мочь отсутствовать для убедительный выражение
телевидение много интересный программа который каждый мочь насладиться небольшой деньга
сравнение другой хобби такой поход театр или посещение спортивный стадион это очень рентабельный хобби который давать зритель несколько косвенный впечатление
однако моей точка зрение телевидение большой плохой чем хороший потому

крупный компания всегда искать новый сотрудник чтобы удовлетворить они потребность распространение управление продажа своей продукция разный страна
например если крупный компания решить открыть филиал другой страна еи потребоваться большой сотрудник этой страна для работа там
другой слово крупный компания означать большой возможностея для трудоустроиство
второй считать что крупный компания мочь давать хороший зарплата чем маленький
что компания крупный значит иметь немалый успех
другой слово это означать что компания мочь увеличить заработный плата свой сотрудник
например здравыя смысл подсказывать мы что известный известный весь мир компания быть давать такой зарплата небольшой местный компания
третий любить путешествовать мир
крупный компания иметь международный репутация организовать международный конференция встреча семинар который дать возможность путешествовать разный страна знакомиться новый человек
также получить новый навык своей область этот семинар
заключение мочь уверенност

третий некоторый родитель опасаться что подработка повлиять учёба студент
сам дело большинство студент быть работать неполный рабочий день время каникулы поэтому это повлиять они учёба
сравнение ежедневноить игрой компьютер подработка лучшие выбор для студент чтобы провести каникулы
заключение подработка мочь позволить учиться испытать настоящий взрослый жизнь узнать нея большой
такой образ родитель должный поощрять студент работать неполный рабочий день
успех учёба зависеть ряд различный фактор один который являться тип работа который склонный выбирать время учёба
частность мы мочь предпочесть работать либо команда другой студент либо одиночка
что касаться предпочитать последний следующий причина
начать тот что тип темперамент один причина которой выбирать индивидуальный работа вместо командноить
быть натура интроверт приходиться тратить много время сила чтобы приспособиться окружающей среда особенно новый человек
однажды когда работать над презентацией незнакомый человек сидеть сторо

например старшея школа человек мочь устоять перед кропотливой учебой день ночь когда другой поразительно амбициозный нацелить престижный университет весь мир делать самый
психологической точка зрение ощущение подчинённость сверстник оказывать детея огромный давление который впоследствии своеобразный стимулятор повышать успеваемость детея
общий когда дело доходить до знание они приобретение одноклассник иметь себя равный
дело тот что ученик проводить вместе даже большой время чем свой родитель объяснять сколько мудрость они усваивать друг друг это мочь быть например урок класс социальный навык или общий правило этикет который родитель иногда упускать вид
ребёнок один возраст наиболее комфортно обмениваться идея исследовать интерпретировать абстрактный понятие опасаться родительский наказание что приводить более глубокий понимание
учитывать преимущество одноклассник иметь себя равный для родителей детея поскольку они обеспечивать огромный стимул расширение кругозор два важнеинуть фактор 

In [48]:
print('Amount of unigrams:',len(unigrams_essays))

Amount of unigrams: 6990


In [ ]:
unigrams_essays

In [56]:
essays_russian_features['bag_of_words'].array[0]

Counter({'всегда': 1,
         'говорить': 1,
         'что': 1,
         'конкуренция': 1,
         'мочь': 1,
         'эффективно': 1,
         'способствовать': 1,
         'развитие': 1,
         'экономика': 1})

In [445]:
essays_russian_ClaimPremise = pd.merge(essays_final,essays_russian,on='Sentence',how='inner')

In [396]:
#russian_essays = translate_chunck(1000,all_sentences_essays, -1)

### td idf vectorizer

In [ ]:
tf_idf_array = []
    corpus = sub_df['array_of_words'].array
    corpus_with_topic = []
    for sent in corpus:
        corpus_with_topic.append(sent)
    corpus_with_topic.append(i)
    #print(i)
    amount_of_sentences = len(corpus)
    #print(amount_of_sentences)
    #print(len(corpus_with_topic))
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus_with_topic)
    array = vectorizer.get_feature_names_out()
    for index in range(len(corpus)):
        tf_idf_vector = X.getrow(index).toarray()[0].ravel()
        tf_idf_array.append(tf_idf_vector)

In [324]:
df_essays.to_csv(r'C:\Users\Маргарита\Desktop\Passau Uni\Магистерская диссертация\DF_ESSAYS.csv', sep=',')

### ECHR

In [590]:
def importdata(filename):
    # open file in reading
    with open(filename, 'r', encoding="utf8") as fr:
        # read from file
        cases = json.load(fr)
    #json to dataframe
    df_all = pd.DataFrame.from_dict(cases)
    return df_all

In [591]:
echr_cases = importdata('ECHR_Corpus.json')
echr_cases 

,name,text,clauses,arguments
0,00.txt,\r\n AS TO THE ADMISSIBIL...,"[{'_id': '5d3fbf313e582511aa1cc855', 'start': ...","[{'premises': ['5d4be09d3e582511aa1cda28', '5d..."
1,01.txt,AS TO THE ADMISSIBILITY ...,"[{'_id': '5d3aae5d3e582511aa1cc155', 'start': ...","[{'premises': ['5d3aae5d3e582511aa1cc155', '5d..."
2,02.txt,\r\n\r\n\r\n AS TO THE ADMISS...,"[{'_id': '5d3a418b3e582511aa1cbe97', 'start': ...","[{'premises': ['5d3a43723e582511aa1cbea0', '5d..."
3,03.txt,\r\n\r\n AS TO THE ADMISS...,"[{'_id': '5d3a5dc43e582511aa1cbf1e', 'start': ...","[{'premises': ['5d3a6cce3e582511aa1cbf29', '5d..."
4,04.txt,\r\n\r\n AS TO THE ADMISS...,"[{'_id': '5d3a85013e582511aa1cbff3', 'start': ...","[{'premises': ['5d3a964d3e582511aa1cc06d', '5d..."
5,05.txt,\r\n\r\n AS TO THE ADMISS...,"[{'_id': '5d3a9a133e582511aa1cc075', 'start': ...","[{'premises': ['5d3a9bc53e582511aa1cc07b'], 'c..."
6,06.txt,\r\n\r\n AS TO THE ADMISS...,"[{'_id': '5d3ab9ac3e582511aa1cc1c1', 'start': ...","[{'premises': ['5d3fa5523e582511aa1cc842'], 'c..."
7,12.txt,\r\n\r\n\r\nCOURT (CHAMBER)\r\n\r\n\r\n\r\n\r\...,"[{'_id': '5d3d1d073e582511aa1cc33b', 'start': ...","[{'premises': ['5d3e5d923e582511aa1cc69c', '5d..."
8,08.txt,\r\n\r\n\r\nCOURT (CHAMBER)\r\n\r\n\r\n\r\n\r\...,"[{'_id': '5d3ac3023e582511aa1cc21d', 'start': ...","[{'premises': ['5d3d7c093e582511aa1cc5d4', '5d..."
9,09.txt,\r\n\r\nFOURTH SECTION\r\n\r\n\r\nCASE OF ALTU...,"[{'_id': '5d3ac88b3e582511aa1cc290', 'start': ...","[{'premises': ['5d3db77b3e582511aa1cc64c', '5d..."


In [592]:
#Просматриваем все столбцы по одной строке
for i in echr_cases :
    print(i, "\n",echr_cases[i][0])

name 
 00.txt
text 
 
                      AS TO THE ADMISSIBILITY OF

                      Application No. 18877/91
                      by Sadik AHMET
                      against Greece


The European Commission of Human Rights sitting in private on 1st July 1994, the following members being present:

           MM.   C.A. NØRGAARD, President
                 F. ERMACORA
                 A.S. GÖZÜBÜYÜK
                 H.G. SCHERMERS
                 H. DANELIUS
                 F. MARTINEZ
                 C.L. ROZAKIS
                 L. LOUCAIDES
                 J.-C. GEUS
                 M.P. PELLONPÄÄ
                 B. MARXER
                 G.B. REFFI
                 M.A. NOWICKI
                 B. CONFORTI
                 I. BÉKÉS
                 E. KONSTANTINOV
                 D. SVÁBY
                 G. RESS

           Mr.   H.C. KRÜGER, Secretary to the Commission

Having regard to Article 25 of the Convention for the Protection of Human Rights and Fundamen

In [593]:
def get_string(start, end, echr_cases, case_number):
    text = echr_cases['text'][case_number]
    return text[start:end]

In [594]:
clauses = pd.DataFrame()
for index in range(len(df_all.index)):
    new_df = pd.DataFrame(df_all['clauses'][index])
    new_df['case_number'] = index
    clauses = pd.concat([clauses, new_df])
clauses

,_id,start,end,case_number
0,5d3fbf313e582511aa1cc855,8903,8951,0
1,5d3fbf383e582511aa1cc856,182,297,0
2,5d3fbf3d3e582511aa1cc857,917,1023,0
3,5d3fbf413e582511aa1cc858,1027,1179,0
4,5d3fbf443e582511aa1cc859,1181,1198,0
...,...,...,...,...
364,5f95b617bf181507836fa55e,63489,63727,41
365,5f95b61cbf181507836fa55f,63728,63797,41
366,5f95b62dbf181507836fa560,63798,64209,41
367,5f95b6e8bf181507836fa561,72196,72245,41


In [595]:
clauses['sentence'] = clauses.apply(lambda row: get_string(row['start'], row['end'],echr_cases, row['case_number']), axis=1)
clauses

,_id,start,end,case_number,sentence
0,5d3fbf313e582511aa1cc855,8903,8951,0,The applicant was also present at the hearing....
1,5d3fbf383e582511aa1cc856,182,297,0,The European Commission of Human Rights sittin...
2,5d3fbf3d3e582511aa1cc857,917,1023,0,Having regard to Article 25 of the Convention ...
3,5d3fbf413e582511aa1cc858,1027,1179,0,Having regard to the application introduced on...
4,5d3fbf443e582511aa1cc859,1181,1198,0,Having regard to:
...,...,...,...,...,...
364,5f95b617bf181507836fa55e,63489,63727,41,"However, one may safely assume that, as a rule..."
365,5f95b61cbf181507836fa55f,63728,63797,41,"Moreover, applicants too must be presumed to b..."
366,5f95b62dbf181507836fa560,63798,64209,41,The onus of alleging and establishing that the...
367,5f95b6e8bf181507836fa561,72196,72245,41,See paragraphs 10 and 15 of the Court's judgment.


In [596]:
arguments = pd.DataFrame()
for index in range(len(df_all.index)):
    for index2 in range(len(df_all['arguments'][index])):
        new_df = pd.DataFrame(df_all['arguments'][index][index2])
        arguments = pd.concat([arguments, new_df])
arguments

,premises,conclusion
0,5d4be09d3e582511aa1cda28,5d4be0d93e582511aa1cda2f
1,5d4be0c33e582511aa1cda2b,5d4be0d93e582511aa1cda2f
0,5d4be0803e582511aa1cda22,5d4be12d3e582511aa1cda3f
1,5d4be0ff3e582511aa1cda37,5d4be12d3e582511aa1cda3f
2,5d4be1143e582511aa1cda3a,5d4be12d3e582511aa1cda3f
...,...,...
0,5f943a99bf181507836fa24c,5f943ab1bf181507836fa24d
0,5f943ad3bf181507836fa24e,5f943aeabf181507836fa24f
0,5f943b0bbf181507836fa250,5f943b22bf181507836fa251
0,5f943c36bf181507836fa253,5f943d0abf181507836fa254


In [597]:
#outer join
echr_final = pd.merge(clauses,arguments,left_on='_id',right_on='premises',how='left')

In [598]:
echr_final.fillna('', inplace=True)

In [599]:
#new_df[new_df['conclusion'].notnull()]

In [600]:
echr_final["type_ADU"] = echr_final.apply(lambda row: 'Premise' if row['conclusion']!='' else '', axis = 1)
echr_final

,_id,start,end,case_number,sentence,premises,conclusion,type_ADU
0,5d3fbf313e582511aa1cc855,8903,8951,0,The applicant was also present at the hearing....,,,
1,5d3fbf383e582511aa1cc856,182,297,0,The European Commission of Human Rights sittin...,,,
2,5d3fbf3d3e582511aa1cc857,917,1023,0,Having regard to Article 25 of the Convention ...,,,
3,5d3fbf413e582511aa1cc858,1027,1179,0,Having regard to the application introduced on...,,,
4,5d3fbf443e582511aa1cc859,1181,1198,0,Having regard to:,,,
...,...,...,...,...,...,...,...,...
10451,5f95b617bf181507836fa55e,63489,63727,41,"However, one may safely assume that, as a rule...",,,
10452,5f95b61cbf181507836fa55f,63728,63797,41,"Moreover, applicants too must be presumed to b...",,,
10453,5f95b62dbf181507836fa560,63798,64209,41,The onus of alleging and establishing that the...,,,
10454,5f95b6e8bf181507836fa561,72196,72245,41,See paragraphs 10 and 15 of the Court's judgment.,,,


In [601]:
conclusions = echr_final["conclusion"].unique()[1:]

In [602]:
echr_final["type_ADU"] = echr_final.apply(lambda row: 'Claim' if row['_id'] in conclusions else row["type_ADU"], axis = 1)
echr_final[echr_final["type_ADU"]!='']

,_id,start,end,case_number,sentence,premises,conclusion,type_ADU
51,5d4be0803e582511aa1cda22,9175,9447,0,Article 5 para. 1 (Art. 5-1) of the Conventio...,5d4be0803e582511aa1cda22,5d4be12d3e582511aa1cda3f,Premise
52,5d4be09d3e582511aa1cda28,9606,9742,0,The Commission notes that the applicant was de...,5d4be09d3e582511aa1cda28,5d4be0d93e582511aa1cda2f,Premise
53,5d4be0c33e582511aa1cda2b,9743,9871,0,He was released after the Court of Appeal revi...,5d4be0c33e582511aa1cda2b,5d4be0d93e582511aa1cda2f,Premise
54,5d4be0d93e582511aa1cda2f,9872,10056,0,The Commission finds that the applicant was de...,,,Claim
55,5d4be0ff3e582511aa1cda37,9872,10056,0,The Commission finds that the applicant was de...,5d4be0ff3e582511aa1cda37,5d4be12d3e582511aa1cda3f,Premise
...,...,...,...,...,...,...,...,...
10228,5f943b22bf181507836fa251,74027,74085,41,For these reasons I find that there has been a...,,,Claim
10229,5f943c36bf181507836fa253,76036,76125,41,(see the Guzzardi v. Italy judgment of 6 Novem...,5f943c36bf181507836fa253,5f943d0abf181507836fa254,Premise
10230,5f943d0abf181507836fa254,74383,74511,41,that the Court cannot deal with the merits of ...,,,Claim
10231,5f943d86bf181507836fa255,75917,76264,41,Article 27 of the Convention (art. 27) require...,5f943d86bf181507836fa255,5f943dabbf181507836fa256,Premise


In [633]:
echr_final[echr_final["type_ADU"]=='Premise']

,_id,start,end,case_number,sentence,premises,conclusion,type_ADU
51,5d4be0803e582511aa1cda22,9175,9447,0,Article 5 para. 1 (Art. 5-1) of the Conventio...,5d4be0803e582511aa1cda22,5d4be12d3e582511aa1cda3f,Premise
52,5d4be09d3e582511aa1cda28,9606,9742,0,The Commission notes that the applicant was de...,5d4be09d3e582511aa1cda28,5d4be0d93e582511aa1cda2f,Premise
53,5d4be0c33e582511aa1cda2b,9743,9871,0,He was released after the Court of Appeal revi...,5d4be0c33e582511aa1cda2b,5d4be0d93e582511aa1cda2f,Premise
55,5d4be0ff3e582511aa1cda37,9872,10056,0,The Commission finds that the applicant was de...,5d4be0ff3e582511aa1cda37,5d4be12d3e582511aa1cda3f,Premise
56,5d4be1143e582511aa1cda3a,10065,10193,0,The Commission also finds no evidence in the c...,5d4be1143e582511aa1cda3a,5d4be12d3e582511aa1cda3f,Premise
...,...,...,...,...,...,...,...,...
10223,5f943a99bf181507836fa24c,74095,74242,41,In its report referred to in paragraph 4 of t...,5f943a99bf181507836fa24c,5f943ab1bf181507836fa24d,Premise
10225,5f943ad3bf181507836fa24e,73730,73887,41,"In sum, I have not been convinced that the ap...",5f943ad3bf181507836fa24e,5f943aeabf181507836fa24f,Premise
10227,5f943b0bbf181507836fa250,73888,74020,41,It follows that neither have I been convinced ...,5f943b0bbf181507836fa250,5f943b22bf181507836fa251,Premise
10229,5f943c36bf181507836fa253,76036,76125,41,(see the Guzzardi v. Italy judgment of 6 Novem...,5f943c36bf181507836fa253,5f943d0abf181507836fa254,Premise


In [625]:
echr_final_test = echr_final[echr_final["type_ADU"]=='Claim']
echr_final_test[echr_final["case_number"]==0]

<ipython-input-625-ab965f0aea21>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  echr_final_test[echr_final["case_number"]==0]


,_id,start,end,case_number,sentence,premises,conclusion,type_ADU
54,5d4be0d93e582511aa1cda2f,9872,10056,0,The Commission finds that the applicant was de...,,,Claim
57,5d4be12d3e582511aa1cda3f,10501,10666,0,It follows that this part of the application i...,,,Claim
60,5d4be1be3e582511aa1cda4a,10500,10664,0,It follows that this part of the application ...,,,Claim
62,5d4be2253e582511aa1cda4c,11038,11186,0,It follows that this part of the application ...,,,Claim
64,5d4be25d3e582511aa1cda4e,10758,10833,0,an unfair hearing in the determination of the ...,,,Claim
69,5d4be2f03e582511aa1cda57,11466,11753,0,"The appliacnt invokes Article 6 paras. 1, 2, 3...",,,Claim
71,5d4be34c3e582511aa1cda65,13023,13196,0,It follows that this part of the application ...,,,Claim
77,5d4be4393e582511aa1cda89,13023,13196,0,It follows that this part of the application ...,,,Claim
79,5d4be4b73e582511aa1cda96,12303,12430,0,The complaint made before the Commission must ...,,,Claim
86,5d4be5683e582511aa1cdab5,13023,13196,0,It follows that this part of the application ...,,,Claim


### AAAI 2020
Automatically retrieved Wikipedia sentences with manual crowd-sourcing scoring

In [17]:
test_data = pd.read_csv('wikipedia_evidence_dataset_29429.csv')
len(test_data['Wikipedia article_title'].unique())

16029

In [18]:
test_data = test_data[test_data['acceptanceRate']>0.9]
len(test_data['Wikipedia article_title'].unique())

1019

In [19]:
test_data

,Motion Text,Dominant Concept,Evidence,acceptanceRate,Wikipedia article_id,Wikipedia article_title
14,We should legalize doping in sport,Doping in sport,"In 2005, the Illicit Drug policy[REF][REF] was...",0.916667,3391589,AFL Players Association
38,We should legalize doping in sport,Doping in sport,Although the extremely dangerous side-effects ...,1.000000,860860,Doping in sport
77,We should legalize doping in sport,Doping in sport,"It is estimated that around 10,000 former athl...",1.000000,860860,Doping in sport
80,We should legalize doping in sport,Doping in sport,A recent study has also shown that long term A...,1.000000,19218324,Anabolic steroid
88,We should legalize doping in sport,Doping in sport,"Opponents claim that with doping legal, all co...",0.928571,860860,Doping in sport
...,...,...,...,...,...,...
29254,We should further exploit sustainable energies,Sustainable energy,"In November 2011, an IEA report entitled Deplo...",1.000000,285292,International Energy Agency
29290,We should further exploit wind turbines,Wind turbine,The Red Hills Wind Farm includes 82 wind turbi...,1.000000,130039,"Elk City, Oklahoma"
29310,We should further exploit wind turbines,Wind turbine,Researchers at the University of Alaska Fairba...,1.000000,369526,Single-wire earth return
29356,We should ban corporal punishment in the home,Corporal punishment in the home,Straus adduced evidence for the view that expo...,1.000000,1737582,School violence


In [ ]:
#Чтобы этот набор данных использовать, нужно подгружать википедия артеклс

### EMNLP 2015
Pre-selected Wikipedia articles

In [20]:
#delete dots which are not at the end of a sentence
def delete_not_sentence_dot(string):
    #By names 
    string = re.sub("[A-Z]\.[A-Z]\.\s+","", string)
    #by abbreviation
    array = re.findall("\.\s+[a-z]", string)
    for change in array:
        string=string.replace(change, change[1:])
    return string
#delete all [REF]
def delete_references(string):
    return string.replace('[REF]','').replace('[REF', '').replace('[RE', '')
#define_article
def define_article(topic, sentence, articles):
    topic_articles = articles[articles['Topic']==topic]
    #print(topic_articles['article Id'].unique())
    full_article_array = topic_articles['full_article'].array
    #print(len(full_article_array))
    for index, article_text in enumerate(full_article_array):
        if article_text.find(sentence)>-1:
            #print(index)
            article_id = topic_articles.iloc[index]['article Id']
            #print(article_id)
            return article_id

In [21]:
#import EMNLP 2015
articles_2 = pd.read_csv('Data4. EMNLP-2015/articles.txt', sep='\t', engine='python', header=None)
#First row is headers row
articles_2.columns = articles_2.iloc[0]
articles_2 = articles_2.drop([0])
claims = pd.read_csv('Data4. EMNLP-2015/claims.txt', sep='\t', engine='python', header=None)
#First row is headers row
claims.columns = claims.iloc[0]
claims = claims.drop([0])
evidence = pd.read_csv('Data4. EMNLP-2015/evidence.txt', sep='\t', engine='python', header=None)
#make columns
evidence.columns = ['Topic', 'Claim', 'Premise', 'Type']
evidence = evidence.drop(columns = ['Type'],axis = 1)

In [22]:
articles_2[articles_2['Topic'] == 'This house would disband ASEAN']

,Topic,Title,article Id
517,This house would disband ASEAN,Association of Southeast Asian Nations,3101
518,This house would disband ASEAN,ASEAN Free Trade Area,3129


In [23]:
evidence[evidence['Topic'] == 'This house would disband ASEAN'].head()

,Topic,Claim,Premise
4476,This house would disband ASEAN,Aside from improving each member state's econo...,"On 15 December 1995, the Southeast Asian Nucle..."
4477,This house would disband ASEAN,Aside from improving each member state's econo...,Head of the International Institute of Strateg...
4478,This house would disband ASEAN,ASEAN has subscribed to the notion of democrat...,"Through the Bali Concord II in 2003, ASEAN has..."
4479,This house would disband ASEAN,ASEAN Way has recently proven itself relativel...,Head of the International Institute of Strateg...
4480,This house would disband ASEAN,ASEAN is criticised for being too soft in its ...,some academics continue to argue that ASEAN's ...


In [24]:
#import full texts of articles
folder_name = 'Data4. articles'
full_articles_2 = {}
for filename in os.listdir(folder_name):
    article_num = int(filename.replace('clean_','').replace('.txt',''))
    with open(os.path.join(folder_name, filename), 'r', encoding="utf8") as f:
        text = f.read()
        full_articles_2[article_num] = text

In [25]:
#add full article to the main dataframe
articles_2['full_article'] = articles_2.apply(lambda row: full_articles_2[int(row['article Id'])], axis=1)

In [26]:
#Function to create dataframe for argumentative/non-argumentation classification
df_premises = evidence.drop(columns = ['Claim'],axis = 1)
df_claims = evidence.drop(columns = ['Premise'],axis = 1)
len(df_premises['Premise'].unique())

2589

In [27]:
#delete duplicates
df_claims = df_claims.drop_duplicates(keep='first')
df_premises = df_premises.drop_duplicates(keep='first')
#add type
df_premises['Type'] = 'Premise'
df_claims['Type'] = 'Claim'
df_premises

,Topic,Premise,Type
0,This house believes that the sale of violent v...,A 2001 study found that exposure to violent vi...,Premise
1,This house believes that the sale of violent v...,The most recent large scale meta-anlysis-- exa...,Premise
2,This house believes that the sale of violent v...,One study did find an increase in reports of b...,Premise
3,This house believes that the sale of violent v...,when one combines all relevant empirical studi...,Premise
4,This house believes that the sale of violent v...,Research published in 2001 suggested that viol...,Premise
...,...,...,...
4686,This house would enforce term limits on the le...,Richard Henry Lee viewed the absence of legal ...,Premise
4687,This house would enforce term limits on the le...,"The historian Mercy Otis Warren, warned that ""...",Premise
4689,This house would enforce term limits on the le...,"James Fenimore Cooper, the novelist, described...",Premise
4690,This house would enforce term limits on the le...,"As of 2002, U.S. Term Limits found that in the...",Premise


In [28]:
#rename columns
df_premises.columns = ['Topic', 'Sentence', 'Type']
df_claims.columns = ['Topic', 'Sentence', 'Type']
#union claims and premises dataframes for using in argumentative/non-argumentative classification
df_ADU_class = pd.concat([df_premises, df_claims])
df_ADU_class['ADU'] = True
#Clean sentence from dots which are not the end of sentence + from [REF]
df_ADU_class['Sentence_clean'] = df_ADU_class.apply(lambda row: delete_references(row['Sentence']), axis=1)
df_ADU_class['Sentence_clean'] = df_ADU_class.apply(lambda row: delete_not_sentence_dot(row['Sentence_clean']), axis=1)

In [29]:
sentences_test = df_ADU_class['Sentence_clean'].array
for sentence in sentences_test:
    print(sentence,'\n')

A 2001 study found that exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world 

The most recent large scale meta-anlysis-- examining 130 studies with over 130,000 subjects worldwide-- concluded that exposure to violent video games causes both short term and long term aggression in players 

One study did find an increase in reports of bullying, noting, "Our research found that certain patterns of video game play were much more likely to be associated with these types of behavioral problems 

when one combines all relevant empirical studies using meta-analytic techniques it shows that violent video games are significantly associated with: increased aggressive behavior, thoughts, and affect; increased physiological arousal; and decreased pro-social (helping) behavior." 

Research published in 2001 suggested that violent video games may increase mild forms of aggressive behavior in children an


In 2009, Princeton sociologist Thomas Espenshade and researcher Alexandria Walton Radford, in their book 'No Longer Separate, Not Yet Equal', examined data on students applying to college in 1997 and calculated that Asian-Americans needed nearly perfect SAT scores of 1550 to have the same chance of being accepted at a top private university as whites who scored 1410 and African-Americans who got 1100. Whites were three times, Hispanics six times, and blacks more than 15 times as likely to be accepted at a US university as Asian-Americans 

opponents of affirmative action, like Ward Connerly, call it reverse discrimination, saying affirmative action requires the very discrimination it is seeking to eliminate 

UCLA professor Richard H. Sander published an article in the November 2004 issue of the Stanford Law Review that questioned the effectiveness of affirmative action in law schools and noted that prior to his article, there had been no comprehensive study on the effects of affirmat

Studies by Keith Ward show that overall religion is a positive contributor to mental health, 

a meta-analysis of 34 recent studies published between 1990 and 2001 also found that religiosity has a salutary relationship with psychological adjustment, being related to less psychological distress, more life satisfaction, and better self-actualization  

a recent systematic review of 850 research papers on the topic concluded that "the majority of well-conducted studies found that higher levels of religious involvement are positively associated with indicators of psychological well-being (life satisfaction, happiness, positive affect, and higher morale) and with less depression, suicidal thoughts and behavior, drug/alcohol use/abuse." 

Some scientific studies show that the degree of religiosity is generally found to be associated with higher ethical attitudes — for example, surveys suggesting a positive connection between faith and altruism  

Moreover, a comprehensive study by Harvard U

Author and investigative journalist David Strahan that same year stated that IEA figures show that carbon dioxide emissions from the oil sands are 20% higher than average emissions from the petroleum production  

According to Greenpeace, oil sands industry has been identified as the largest contributor to greenhouse gas emissions growth in Canada, as it accounts for 40&nbsp;million tons of emissions per year  

In 2007, Environment Canada completed a study that shows high deformity rates in fish embryos exposed to the oil sands 

David W. Schindler, a limnologist from the University of Alberta, co-authored a study on Alberta's oil sands' contribution of aromatic polycyclic compounds, some of which are known carcinogens, to the Athabasca River and its tributaries  

for instance, moose were found in a 2006 study to have as high as 453 times the acceptable levels of arsenic in their systems 

The market size for oil and gas pipeline construction experienced tremendous growth prior to th

David Walsh, from the National Institute on Media and the Family, thinks that when adolescents engage in casual sexual relationships they do not develop skills such as trust and communication that are key ingredients in healthy, long-lasting relationships  

In purely sexual relationships, adolescents pick up "a lot of bad habits" and don't learn "to trust or share or know how to disagree and make up", according to Laura Sessions Stepp, author of Unhooked: How Young Women Pursue Sex, Delay Love, and Lose at Both . They become jaded and as a result later in life, they have trouble forming adult relationships, according to Levy-Warren . "They don't learn to build that emotional intimacy before they get physically intimate. In the long term, that develops bad relationship habits," according to adolescent gynecologist Melisa Holmes, author of Girlology: Hang-Ups, Hook-Ups and Holding Out 

Experts worry that when teens have sex before they're ready then they undervalue the experience and t

...

On 5 March 2008, an amendment was passed to the Criminal Justice and Immigration Act 2008 which abolished the common law offences of blasphemy and blasphemous libel in England and Wales 

Eileen Donahoe, the US ambassador, also rejected the resolution. She said, "We cannot agree that prohibiting speech is the way to promote tolerance, because we continue to see the 'defamation of religions' concept used to justify censorship, criminalisation, and in some cases violent assaults and deaths of political, racial, and religious minorities around the world." 

In August 2007, the Special Rapporteur, Doudou Diène, reported to the General Assembly "on contemporary forms of racism, racial discrimination, xenophobia and related intolerance on the manifestations of defamation of religions and in particular on the serious implications of Islamophobia on the enjoyment of all rights 

The OIC's representative on the UNHRC, Zamir Akram (Pakistan), said, "Negative stereotyping or defamation of religio

Too much exercise can be harmful 

children who fail to engage in regular physical activity are at greater risk of obesity 

Physical inactivity as a child could result in physical inactivity as an adult 

Childhood inactivity is linked to obesity 

the model of ideal education is that which occurs when people go on their own initiative to discover things 

Physical inactivity is increasing or high among many groups in the population 

people die as a result of physical inactivity 

people can improve their health via exercise 

a lack of physical activity is thought to explain most cases of obesity 

A sedentary lifestyle plays a significant role in obesity  

Excess weight has reached epidemic proportions globally 

actually increases test scores 

youth do not get the physical outlet needed not only for their cognitive development but for their physical health 

school should be made to fit the child, rather than the other way around 

children learn best with freedom from coercion 

foreign aid is efficacious 

in many instances, aid is conditionally tied due to political motives, rather than notions of proper policy and implementation 

When a country is given money because they cannot financially sustain themselves, several negative effects have the potential to develop 

Foreign aid simply does not work 

pouring vast amounts of money into development aid without any concern for results has failed 

Trade is a key factor in economic development 

A successful use of trade can boost a country's development 

trade, development, and poverty reduction are intimately linked 

trade and growth are strongly linked 

export-led growth has been a key part of many countries’ successful development strategies 

trade-led growth is pro-poor 

foreign aid to many third world countries has failed to produce sustainable growth 

sufficient foreign aid can make up for the lack of capital in poor countries 

If the foreign assistance is substantial enough, and lasts long enoug

Circumcision provides important health advantages which outweigh the risks 

non-therapeutic circumcision of male minors is a violation of children’s rights to autonomy and physical integrity 

Circumcision remains a relatively safe procedure 

the overall effect of male neonatal non-therapeutic circumcision on health is more likely to be negative rather than positive 

complications from bleeding, infection and poorly carried out circumcisions can be catastrophic 

newborn circumcisions are a significant source of pain 

circumcision causes severe pain 

The removal of trees without sufficient reforestation has resulted in damage to habitat 

The removal of trees without sufficient reforestation has adverse impacts on biosequestration of atmospheric carbon dioxide 

Deforestation causes extinction 

Deforestation is a contributor to global warming 

Deforestation is often cited as one of the major causes of the enhanced greenhouse effect 

deforestation will have no effect on atmosphe

The use of (or threat of use of) nuclear weapons would generally be contrary to the rules of international law applicable in armed conflict 

The spread of nuclear weapons can increase international stability 

Nuclear proliferation will decrease the likelihood of war 

The effective prohibition on nuclear proliferation has been characterised as a form of technological apartheid 

nuclear forces continue to play an essential role in war prevention 

nuclear weapons can induce stability 

controlled nuclear proliferation may be beneficial for inducing stability 

Nuclear proliferation increases the chance of inter-state nuclear conflict 

Nuclear proliferation increases the chances of nuclear material falling into the hands of non-state groups 

new nuclear states will use their acquired nuclear capabilities to deter threats and preserve peace 

new nuclear states often lack adequate organizational controls over their new weapons 

nuclear weapons induce stability in all regimes 

the p

In [30]:
df_ADU_class

,Topic,Sentence,Type,ADU,Sentence_clean
0,This house believes that the sale of violent v...,A 2001 study found that exposure to violent vi...,Premise,True,A 2001 study found that exposure to violent vi...
1,This house believes that the sale of violent v...,The most recent large scale meta-anlysis-- exa...,Premise,True,The most recent large scale meta-anlysis-- exa...
2,This house believes that the sale of violent v...,One study did find an increase in reports of b...,Premise,True,One study did find an increase in reports of b...
3,This house believes that the sale of violent v...,when one combines all relevant empirical studi...,Premise,True,when one combines all relevant empirical studi...
4,This house believes that the sale of violent v...,Research published in 2001 suggested that viol...,Premise,True,Research published in 2001 suggested that viol...
...,...,...,...,...,...
4679,This house believes social deprivation causes ...,an environment of great inequalities between p...,Claim,True,an environment of great inequalities between p...
4682,This house would enforce term limits on the le...,"The longer politicians are in office, the more...",Claim,True,"The longer politicians are in office, the more..."
4683,This house would enforce term limits on the le...,The lack of mandatory limits to tenure is rega...,Claim,True,The lack of mandatory limits to tenure is rega...
4688,This house would enforce term limits on the le...,contact with the affairs of state is one of th...,Claim,True,contact with the affairs of state is one of th...


In [31]:
len(df_ADU_class['Sentence'].unique())

3958

In [32]:
len(df_ADU_class['Topic'].unique())

58

In [33]:
articles_2

,Topic,Title,article Id,full_article
1,This house believes that the sale of violent v...,Video game content rating system,567,A video game content rating system is a system...
2,This house believes that the sale of violent v...,Grand Theft Childhood,524,Grand Theft Childhood: The Surprising Truth Ab...
3,This house believes that the sale of violent v...,School violence,542,School violence is widely held to have become ...
4,This house believes that the sale of violent v...,Video game controversies,1,. \n\nControversies over video games often cen...
5,This house believes that the sale of violent v...,Nonviolent video game,503,Nonviolent video games are video games charact...
...,...,...,...,...
543,This house believes social deprivation causes ...,Social disorganization theory,3284,". \n\nIn sociology, the Social disorganization..."
544,This house believes social deprivation causes ...,Violence,3293,Violence is defined by the World Health Organi...
545,This house would enforce term limits on the le...,Term limits in the United States,3163,Term limits in the United States apply to many...
546,This house would enforce term limits on the le...,Howard Rich,3127,". \n\nHoward S. Rich, also Howie Rich[REF] (b...."


In [34]:
#add article id to a sentence
df_ADU_class['article Id'] = df_ADU_class.apply(lambda row: define_article(row['Topic'], row['Sentence'], articles_2), axis = 1)
df_ADU_class_article=df_ADU_class.dropna()
len(df_ADU_class_article['article Id'].unique())

399

In [35]:
df_ADU_class_article[df_ADU_class_article['article Id'] == '567']

,Topic,Sentence,Type,ADU,Sentence_clean,article Id
337,This house believes that the sale of violent v...,It has been shown that action video game playe...,Premise,True,It has been shown that action video game playe...,567
427,This house believes that the sale of violent v...,numerous researchers have proposed potential p...,Claim,True,numerous researchers have proposed potential p...,567


In [36]:
len(df_ADU_class_article['article Id'].unique())

399

In [37]:
len(articles_2['article Id'].unique())

547

In [38]:
#merge
full = df_ADU_class = pd.merge(df_ADU_class,articles_2.drop(['Topic'], axis = 1),on='article Id',how='inner')
full.head()

,Topic,Sentence,Type,ADU,Sentence_clean,article Id,Title,full_article
0,This house believes that the sale of violent v...,A 2001 study found that exposure to violent vi...,Premise,True,A 2001 study found that exposure to violent vi...,1,Video game controversies,. \n\nControversies over video games often cen...
1,This house believes that the sale of violent v...,The most recent large scale meta-anlysis-- exa...,Premise,True,The most recent large scale meta-anlysis-- exa...,1,Video game controversies,. \n\nControversies over video games often cen...
2,This house believes that the sale of violent v...,One study did find an increase in reports of b...,Premise,True,One study did find an increase in reports of b...,1,Video game controversies,. \n\nControversies over video games often cen...
3,This house believes that the sale of violent v...,when one combines all relevant empirical studi...,Premise,True,when one combines all relevant empirical studi...,1,Video game controversies,. \n\nControversies over video games often cen...
4,This house believes that the sale of violent v...,Research published in 2001 suggested that viol...,Premise,True,Research published in 2001 suggested that viol...,1,Video game controversies,. \n\nControversies over video games often cen...


In [39]:
len(full.index)

3424

In [40]:
len(full['article Id'].unique())

399

In [ ]:
# ADUs = article_arguments['Sentence_clean'].array
# ADUs_types = article_arguments['Type'].array
# #print(len(ADUs))
# sentences_adu = {}
# for index, adu in enumerate(ADUs):
#     #print(adu, '\n')
#     separate_sentences = re.split(r"\.\s+|\!|\?", adu)
#     for i in separate_sentences:
#         sentences_adu[i] = ADUs_types[index]
# sentence_adu

# for adu in sentences_adu:
#     for sub_adu in adu:
#         if sentence.find(sub_adu)>-1:
#                         #print('Нашёл предложение')
#                         isADU=True
#                         break

#### get position

In [47]:
df_task_ADU = pd.DataFrame()
for article_id in full['article Id'].unique():
    #print(article_id)
    article_arguments = full[full['article Id']==article_id]
    #get text
    topic = article_arguments.iloc[0]['Topic']
    #print(topic)
    text = article_arguments.iloc[0]['full_article']
    sentences = re.split(r"\.\s+|\!|\?", text)
    #Make claims
    ADUs_claim = article_arguments[article_arguments['Type']=='Claim']['Sentence_clean'].array
    #print(len(ADUs))
    sentences_adu_claim = []
    for adu in ADUs_claim:
        #print(adu, '\n')
        sentences_adu_claim.append(re.split(r"\.\s+|\!|\?", adu))
    #Make premises
    ADUs_premises = article_arguments[article_arguments['Type']=='Premise']['Sentence_clean'].array
    #print(len(ADUs))
    sentences_adu_premises = []
    for adu in ADUs_premises:
        #print(adu, '\n')
        sentences_adu_premises.append(re.split(r"\.\s+|\!|\?", adu))
    #print(sentences_adu)
    #we add constraint x10
    ADUs = article_arguments['Sentence_clean'].array
    constraint = len(ADUs)*10
    #print('Органичение', constraint)
    count_added = 0
    #print(len(sentences))
    for index, sentence in enumerate(sentences):
        #check if a sentence consists of enough words
        words = re.split(r"\s+", sentence)
        if len(words)>4:
        #how can we find sentence
            number_of_sentence = index+1
            sentence_position = number_of_sentence/len(sentences)
            start = text.find(sentence)
            end = start+len(sentence)
            isADU = False
            Type =''
            #for premise
            for adu in sentences_adu_premises:
                for sub_adu in adu:
                    if sentence.find(sub_adu)>-1:
                        #print('Нашёл предложение')
                        isADU=True
                        Type = 'Premise'
                        break
                if isADU:
                    break
            #for claims
            if not isADU:
                for adu in sentences_adu_claim:
                    for sub_adu in adu:
                        if sentence.find(sub_adu)>-1:
                            #print('Нашёл предложение')
                            isADU=True
                            Type = 'Claim'
                            break
                    if isADU:
                        break
            if count_added<=constraint or isADU:
                df_task_ADU = df_task_ADU.append({'Article_id':article_id,'Topic':topic,'Sentence':sentence, 'Type':Type, 'Sentence_number':number_of_sentence, 'Sentence_position':sentence_position,'Start':start, 'End':end,'ADU':isADU}, ignore_index=True)
                if not isADU:
                    count_added+=1
                    #print('Добавилено неаргументов:', count_added)
#print(article_id,':', len(article_arguments.index))

In [1141]:
# df_task_ADU = pd.DataFrame()
# for article_id in full['article Id'].unique():
#     #print(article_id)
#     article_arguments = full[full['article Id']==article_id]
#     #get text
#     topic = article_arguments.iloc[0]['Topic']
#     #print(topic)
#     text = article_arguments.iloc[0]['full_article']
#     sentences = re.split(r"\.\s+|\!|\?", text)
#     ADUs = article_arguments['Sentence_clean'].array
#     #print(len(ADUs))
#     sentences_adu = []
#     for adu in ADUs:
#         #print(adu, '\n')
#         sentences_adu.append(re.split(r"\.\s+|\!|\?", adu))
#     #print(sentences_adu)
#     #we add constraint x10
#     constraint = len(ADUs)*10
#     #print('Органичение', constraint)
#     count_added = 0
#     #print(len(sentences))
#     for index, sentence in enumerate(sentences):
#         #check if a sentence consists of enough words
#         words = re.split(r"\s+", sentence)
#         if len(words)>4:
#         #how can we find sentence
#             number_of_sentence = index+1
#             sentence_position = number_of_sentence/len(sentences)
#             start = text.find(sentence)
#             end = start+len(sentence)
#             isADU = False
#             for adu in sentences_adu:
#                 for sub_adu in adu:
#                     if sentence.find(sub_adu)>-1:
#                         #print('Нашёл предложение')
#                         isADU=True
#                         break
#                 if isADU:
#                     break
#             if count_added<=constraint or isADU:
#                 df_task_ADU = df_task_ADU.append({'Article_id':article_id,'Topic':topic,'Sentence':sentence, 'Sentence_number':number_of_sentence, 'Sentence_position':sentence_position,'Start':start, 'End':end,'ADU':isADU}, ignore_index=True)
#                 if not isADU:
#                     count_added+=1
#                     #print('Добавилено неаргументов:', count_added)
# #print(article_id,':', len(article_arguments.index))

In [1138]:
#df_task_ADU['Sentence']= df_task_ADU.apply(lambda row: delete_references(row['Sentence']), axis = 1)
#test = df_task_ADU[df_task_ADU['ADU']==True]['Sentence']
#for t in test:
#    print(t, '\n')

In [49]:
df_task_ADU[df_task_ADU['ADU']==False]

,ADU,Article_id,End,Sentence,Sentence_number,Sentence_position,Start,Topic,Type
0,0.0,1,248.0,Controversies over video games often center on...,2.0,0.007435,4.0,This house believes that the sale of violent v...,
1,0.0,1,319.0,Video games have been studied for links to add...,3.0,0.011152,252.0,This house believes that the sale of violent v...,
2,0.0,1,392.0,Earlier meta-analyses (an analysis of several ...,4.0,0.014870,321.0,This house believes that the sale of violent v...,
8,0.0,1,1321.0,It has been argued there is generally a lack o...,10.0,0.037175,1116.0,This house believes that the sale of violent v...,
10,0.0,1,1946.0,"However, this meta-analysis was severely criti...",12.0,0.044610,1579.0,This house believes that the sale of violent v...,
...,...,...,...,...,...,...,...,...,...
30755,0.0,2735,2772.0,The Compact City had a particularly strong inf...,18.0,0.310345,2648.0,This house would build high rises for housing,
30756,0.0,2735,2918.0,The first Labour Government in 1998 set up the...,19.0,0.327586,2774.0,This house would build high rises for housing,
30757,0.0,2735,3309.0,"Influenced by this report, the UK Government i...",20.0,0.344828,2920.0,This house would build high rises for housing,
30758,0.0,2735,3486.0,"Over the succeeding years, these targets were ...",21.0,0.362069,3311.0,This house would build high rises for housing,


In [69]:
test_df = translate_chunck(1000,df_task_ADU,5)

<ipython-input-63-378596411060>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Russian_sentence'] = sub_df['Sentence'].apply(google_traslator.translate, src='id', dest='en')


In [ ]:
test_df = translate_chunck(1000,df_task_ADU,5)

In [245]:
test_df_task_ADU = test_df
test_df_task_ADU[test_df_task_ADU['ADU']==True]

,ADU,Article_id,End,Sentence,Sentence_number,Sentence_position,Start,Topic,Type,Russian_sentence,string_length,amount_of_words,bag_of_words,sentence_embedding
3,1.0,1,568.0,A 2001 study found that exposure to violent vi...,5.0,0.018587,394.0,This house believes that the sale of violent v...,Premise,"Исследование 2001 года показало, что воздейств...",182,1,"{'исследование': 1, 'год': 1, 'показать': 1, '...","[0.04837178049557325, 0.05649031377914879, 0.1..."
4,1.0,1,671.0,A decrease in prosocial behavior (caring about...,6.0,0.022305,570.0,This house believes that the sale of violent v...,Premise,Также было отмечено снижение просоциального по...,100,1,"{'также': 1, 'отметить': 1, 'снижение': 1, 'пр...","[0.19291716050356628, 0.041402529925107956, 0...."
5,1.0,1,907.0,Another 2001 meta-analyses using similar metho...,7.0,0.026022,673.0,This house believes that the sale of violent v...,Claim,Другой метаанализ 2001 года с использованием а...,277,1,"{'другой': 1, 'метаанализ': 1, 'год': 2, 'испо...","[0.042621947339047554, -0.00318128386836337, -..."
6,1.0,1,967.0,Many potential positive effects have been prop...,8.0,0.029740,911.0,This house believes that the sale of violent v...,Premise,Было предложено много потенциальных положитель...,64,1,"{'предложить': 1, 'много': 1, 'потенциальный':...","[-0.10237151058390737, 0.16887670258680978, -0..."
7,1.0,1,1112.0,Recent research has suggested that some violen...,9.0,0.033457,969.0,This house believes that the sale of violent v...,Premise,"Недавние исследования показали, что некоторые ...",155,1,"{'недавний': 1, 'исследование': 1, 'показать':...","[-0.07395038823597133, -0.11469371477141976, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4941,1.0,239,2234.0,"By 2001, the estimated number of people who ha...",17.0,0.090909,2026.0,This house would ban gambling,Premise,"К 2001 году оценочное число людей, участвовавш...",165,1,"{'год': 1, 'оценочный': 1, 'число': 1, 'людея'...","[-0.19226072961464524, -0.09387282957322896, -..."
4942,1.0,239,2331.0,"In 2008, H2 Gambling Capital estimates worldwi...",18.0,0.096257,2238.0,This house would ban gambling,Premise,В 2008 году H2 Gambling Capital оценивает миро...,105,1,"{'год': 1, 'gambling': 1, 'capital': 1, 'оцени...","[-0.18642104078422894, -0.016460646959868343, ..."
4945,1.0,239,2700.0,Internet gambling has become one of the most p...,21.0,0.112299,2601.0,This house would ban gambling,Premise,Азартные игры в Интернете стали одним из самых...,97,1,"{'азартный': 1, 'игра': 1, 'интернет': 2, 'ста...","[-0.11355762258172035, 0.06405545845627784, 0...."
4946,1.0,239,2847.0,In 2007 the gambling commission stated that th...,22.0,0.117647,2702.0,This house would ban gambling,Premise,В 2007 году комиссия по азартным играм заявила...,176,1,"{'год': 1, 'комиссия': 2, 'азартный': 3, 'игра...","[-0.05459546397573182, -0.014962376760584968, ..."


### Creating features

In [246]:
test_df_task_ADU['string_length'] = test_df_task_ADU.apply(lambda row: len(row['Russian_sentence']), axis=1)

#### lemmatization

In [256]:
test_df_task_ADU['amount_of_words'] = test_df_task_ADU.apply(lambda row: amount_of_words(row['Russian_sentence']), axis=1)
test_df_task_ADU['bag_of_words'] = test_df_task_ADU.apply(lambda row: create_bag_of_words(row['Russian_sentence']), axis=1)
#надо бы по-другому назвать 
test_df_task_ADU['array_of_words'] = test_df_task_ADU.apply(lambda row: lemmatize_sentence(row['Russian_sentence']), axis=1)
test_df_task_ADU

#### embeddings

In [258]:
def NER_analysis(text):
    markup = ner(text)
    spans = markup.spans
    spantypes_amount = {}
    for span in spans:
        if span.type in spantypes_amount.keys():
            spantypes_amount[span.type] = spantypes_amount[span.type]+1
        else:
            spantypes_amount[span.type] = 1
    return spantypes_amount

In [259]:
def NER_morph_analysis(sentence):
    chunk = []
    tokens = [_.text for _ in tokenize(sentence)]
    chunk.append(tokens)
    markup = next(morph.map(chunk))
    spantypes_amount = {}
    for token in markup.tokens:
        splitted = token.tag.split('|')
        tag = splitted[0]
        if tag in spantypes_amount.keys():
            spantypes_amount[tag] = spantypes_amount[tag]+1
        else:
            spantypes_amount[tag] = 1
    return spantypes_amount

In [260]:
def NER_syntax_analysis(sentence):
    chunk = []
    tokens = [_.text for _ in tokenize(sentence)]
    chunk.append(tokens)
    markup = next(syntax.map(chunk))
    # Convert CoNLL-style format to source, target indices
    words, deps = [], []
    for token in markup.tokens:
        words.append(token.text)
        source = int(token.head_id) - 1
        target = int(token.id) - 1
        if source > 0 and source != target:  # skip root, loops
            deps.append([source, target, token.rel])
    spantypes_amount = {}
    for token in deps:
        tag = token[2]
        if tag in spantypes_amount.keys():
            spantypes_amount[tag] = spantypes_amount[tag]+1
        else:
            spantypes_amount[tag] = 1
    return spantypes_amount

In [261]:
#sentence embedding
def get_sentence_embedding(bag_of_words):
    #Надо сначала добавить просто
    words = list(bag_of_words.keys())
    #print(words)
    sent_embedding = [0.00]*300
    for word in words:
        amount = bag_of_words[word]
        #print(word)
        not_found=False
        try:
            embedding = navec[word]
        except:
            not_found = True
        for i in range(amount):
            #print('До:',len(sent_embedding))
            #print('Эмбеддинг:',len(embedding))
            #print(sent_embedding)
            #print(embedding)
            if not not_found:
                sent_embedding += embedding
                #print('После:',len(sent_embedding))
    amount_of_words = len(words)
    if amount_of_words == 0:
        #print(words)
        sent_embedding = ''
    else:
        sent_embedding = [i/amount_of_words for i in sent_embedding]
    return sent_embedding

In [262]:
#Так как видеоигра важное слово для понимания видеоигра очень важна заменим её на игра 
#sentence.replace['видеоигра', 'игра']

In [263]:
test_df_task_ADU['sentence_embedding'] = test_df_task_ADU.apply(lambda row: get_sentence_embedding(row['bag_of_words']), axis=1)

In [264]:
test_df_task_ADU = test_df_task_ADU.loc[test_df_task_ADU['sentence_embedding']!='']

In [265]:
#add Russian topics
eng_rus_topics = {}
topic_names = articles_2['Topic'].unique()
for topic_name in topic_names:
    topic_name_Russian = google_traslator.translate(topic_name)
    eng_rus_topics[topic_name] = topic_name_Russian

In [266]:
test_df_task_ADU['Russian_topic'] = test_df_task_ADU.apply(lambda row: eng_rus_topics[row['Topic']], axis=1)

<ipython-input-266-c7b82d30baae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_task_ADU['Russian_topic'] = test_df_task_ADU.apply(lambda row: eng_rus_topics[row['Topic']], axis=1)


In [267]:
#add topic embeddings
topic_embeddings = {}
topic_names=test_df_task_ADU['Russian_topic'].unique()
for topic_name in topic_names:
    bag_of_words = create_bag_of_words(topic_name)
    embedding = get_sentence_embedding(bag_of_words)
    topic_embeddings[topic_name] = embedding

In [268]:
test_df_task_ADU['Topic_embedding'] = test_df_task_ADU.apply(lambda row: topic_embeddings[row['Russian_topic']], axis=1)

<ipython-input-268-2c729033bca1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_task_ADU['Topic_embedding'] = test_df_task_ADU.apply(lambda row: topic_embeddings[row['Russian_topic']], axis=1)


In [269]:
#cosine similarity
def cosine_similarity(List1, List2):
    result = dot(List1, List2)/(norm(List1)*norm(List2))
    return result

In [270]:
test_df_task_ADU['Topic_Sentence_Similarity'] = test_df_task_ADU.apply(lambda row: cosine_similarity(row['Topic_embedding'], row['sentence_embedding']), axis=1)

<ipython-input-269-93fded0c85a4>:3: RuntimeWarning: invalid value encountered in double_scalars
  result = dot(List1, List2)/(norm(List1)*norm(List2))
<ipython-input-270-e7fba463073d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_task_ADU['Topic_Sentence_Similarity'] = test_df_task_ADU.apply(lambda row: cosine_similarity(row['Topic_embedding'], row['sentence_embedding']), axis=1)


### NER

In [271]:
test_df_task_ADU['NER_analysis'] = test_df_task_ADU.apply(lambda row: NER_analysis(row['Russian_sentence']), axis=1)
test_df_task_ADU['NER_morph_analysis'] = test_df_task_ADU.apply(lambda row: NER_morph_analysis(row['Russian_sentence']), axis=1)
test_df_task_ADU['NER_syntax_analysis'] = test_df_task_ADU.apply(lambda row: NER_syntax_analysis(row['Russian_sentence']), axis=1)

<ipython-input-271-5619600f2a0e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_task_ADU['NER_analysis'] = test_df_task_ADU.apply(lambda row: NER_analysis(row['Russian_sentence']), axis=1)
<ipython-input-271-5619600f2a0e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_task_ADU['NER_morph_analysis'] = test_df_task_ADU.apply(lambda row: NER_morph_analysis(row['Russian_sentence']), axis=1)
<ipython-input-271-5619600f2a0e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [272]:
test_df_task_ADU

,ADU,Article_id,End,Sentence,Sentence_number,Sentence_position,Start,Topic,Type,Russian_sentence,...,amount_of_words,bag_of_words,sentence_embedding,array_of_words,Russian_topic,Topic_embedding,Topic_Sentence_Similarity,NER_analysis,NER_morph_analysis,NER_syntax_analysis
0,0.0,1,248.0,Controversies over video games often center on...,2.0,0.007435,4.0,This house believes that the sale of violent v...,,Споры вокруг видеоигр часто сосредоточены на т...,...,27,"{'спор': 1, 'вокруг': 1, 'видеоигра': 2, 'част...","[0.06149787470698356, -0.008984958678483963, 0...",спор вокруг видеоигра часто сосредоточить тако...,"Этот дом считает, что продажа жестоких видеоиг...","[-0.018784684129059315, -0.05282188877463341, ...",0.585827,{},"{'NOUN': 14, 'ADP': 3, 'ADV': 1, 'VERB': 1, 'D...","{'nsubj:pass': 1, 'case': 4, 'advmod': 1, 'det..."
1,0.0,1,319.0,Video games have been studied for links to add...,3.0,0.011152,252.0,This house believes that the sale of violent v...,,Видеоигры были изучены на предмет связи с зави...,...,7,"{'видеоигра': 1, 'быть': 1, 'изучить': 1, 'пре...","[0.25010903445737703, -0.0391185099392065, 0.1...",видеоигра быть изучить предмет связь зависимос...,"Этот дом считает, что продажа жестоких видеоиг...","[-0.018784684129059315, -0.05282188877463341, ...",0.444114,{},"{'PROPN': 1, 'AUX': 1, 'VERB': 1, 'ADP': 2, 'N...","{'nsubj:pass': 1, 'aux:pass': 1, 'case': 2, 'o..."
2,0.0,1,392.0,Earlier meta-analyses (an analysis of several ...,4.0,0.014870,321.0,This house believes that the sale of violent v...,,Более ранние метаанализы (анализ нескольких ис...,...,8,"{'более': 1, 'ранний': 1, 'метаанализ': 1, 'ан...","[0.01605249452404678, -0.14018025062978268, 0....",более ранний метаанализ анализ несколько иссле...,"Этот дом считает, что продажа жестоких видеоиг...","[-0.018784684129059315, -0.05282188877463341, ...",0.514264,{},"{'ADV': 1, 'ADJ': 2, 'NOUN': 3, 'PUNCT': 3, 'N...","{'advmod': 1, 'amod': 1, 'nsubj': 1, 'punct': ..."
3,1.0,1,568.0,A 2001 study found that exposure to violent vi...,5.0,0.018587,394.0,This house believes that the sale of violent v...,Premise,"Исследование 2001 года показало, что воздейств...",...,20,"{'исследование': 1, 'год': 1, 'показать': 1, '...","[0.04837178049557325, 0.05649031377914879, 0.1...",исследование год показать что воздеиствие жест...,"Этот дом считает, что продажа жестоких видеоиг...","[-0.018784684129059315, -0.05282188877463341, ...",0.697170,{},"{'NOUN': 10, 'ADJ': 5, 'VERB': 3, 'PUNCT': 2, ...","{'nsubj': 2, 'amod': 4, 'punct': 2, 'mark': 2,..."
4,1.0,1,671.0,A decrease in prosocial behavior (caring about...,6.0,0.022305,570.0,This house believes that the sale of violent v...,Premise,Также было отмечено снижение просоциального по...,...,10,"{'также': 1, 'отметить': 1, 'снижение': 1, 'пр...","[0.19291716050356628, 0.041402529925107956, 0....",также отметить снижение просоциальный поведени...,"Этот дом считает, что продажа жестоких видеоиг...","[-0.018784684129059315, -0.05282188877463341, ...",0.495417,{},"{'ADV': 1, 'AUX': 1, 'VERB': 1, 'NOUN': 5, 'AD...","{'advmod': 1, 'aux:pass': 1, 'nsubj:pass': 1, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,239,9876.0,"At the UIGEA bill-signing ceremony, Bush did n...",74.0,0.395722,9701.0,This house would ban gambling,,На церемонии подписания законопроекта UIGEA Бу...,...,21,"{'церемония': 1, 'подписание': 1, 'законопроек...","[-0.046514085915294434, -0.17127214443115962, ...",церемония подписание законопроект uigea буш уп...,Этот дом запретит азартные игры,"[-0.12897785604000092, -0.15007487088441848, 0...",0.638245,"{'PER': 1, 'ORG': 2}","{'ADP': 4, 'NOUN': 7, 'PROPN': 4, 'PART': 1, '...","{'case': 4, 'obl': 2, 'nmod': 5, 'nsubj': 3, '..."
4996,0.0,239,10115.0,In response to Unlawful Internet Gambling Enfo...,75.0,0.401070,9880.0,This house would ban gambling,,В ответ на Закон о борьбе с незаконными азартн...,...,28,"{'ответ': 1, 'закон': 1, 'борьба': 1, 'незакон...","[-0.08864082252

In [273]:
#from dictionary to columns
def get_columns(array_dict):
    columns = []
    for dic in array_dict:
        keys = dic.keys()
        for key in keys:
            if key not in columns:
                columns.append(key)
    return columns

In [274]:
def dictvalue_or_emphty(dictionary, key):
    #print(dictionary)
    #print(key)
    try:
        value = int(dictionary[key])
        #print(value)
    except:
        value = 0
    return value

In [275]:
#create columns for NER_analysis
additional_columns = get_columns(test_df_task_ADU['NER_analysis'].array)
#print(additional_columns)
for column in additional_columns:
    #print(column)
    test_df_task_ADU[column] = test_df_task_ADU.apply(lambda row: dictvalue_or_emphty(row['NER_analysis'], column), axis=1)
#create columns for NER_morph_analysis
additional_columns = get_columns(test_df_task_ADU['NER_morph_analysis'].array)
for column in additional_columns:
    test_df_task_ADU[column] = test_df_task_ADU.apply(lambda row: dictvalue_or_emphty(row['NER_morph_analysis'], column), axis=1)

<ipython-input-275-b88d9d83d082>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_task_ADU[column] = test_df_task_ADU.apply(lambda row: dictvalue_or_emphty(row['NER_analysis'], column), axis=1)
<ipython-input-275-b88d9d83d082>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_task_ADU[column] = test_df_task_ADU.apply(lambda row: dictvalue_or_emphty(row['NER_morph_analysis'], column), axis=1)


In [276]:
test_df_task_ADU.columns[:10]

Index(['ADU', 'Article_id', 'End', 'Sentence', 'Sentence_number',
       'Sentence_position', 'Start', 'Topic', 'Type', 'Russian_sentence'],
      dtype='object')

In [277]:
#create columns for NER_syntax_analysis
additional_columns = get_columns(test_df_task_ADU['NER_syntax_analysis'].array)
for column in additional_columns:
    test_df_task_ADU[column] = test_df_task_ADU.apply(lambda row: dictvalue_or_emphty(row['NER_syntax_analysis'], column), axis=1)

<ipython-input-277-d1939c6ad216>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_task_ADU[column] = test_df_task_ADU.apply(lambda row: dictvalue_or_emphty(row['NER_syntax_analysis'], column), axis=1)


In [ ]:
#Cosine similarity by words


In [ ]:
#Составлять массив слов без подсчета элементов (не как в bag of words)

In [342]:
#Тут проблема в том, что я беру группирую по русскому названию, а группировать в целом не надо
g = test_df_task_ADU.groupby("Russian_topic")
new_df = pd.DataFrame()
for (i, sub_df) in g:
    tf_idf_array = []
    corpus = sub_df['array_of_words'].array
    corpus_with_topic = []
    for sent in corpus:
        corpus_with_topic.append(sent)
    corpus_with_topic.append(i)
    #print(i)
    amount_of_sentences = len(corpus)
    #print(amount_of_sentences)
    #print(len(corpus_with_topic))
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus_with_topic)
    array = vectorizer.get_feature_names_out()
    for index in range(len(corpus)):
        tf_idf_vector = X.getrow(index).toarray()[0].ravel()
        tf_idf_array.append(tf_idf_vector)
    u = g.get_group(i)
    #print(len(tf_idf_array))
    #print(len(u.index))
    u['tf_idf_vector'] = tf_idf_array
    topic_vector = X.getrow(amount_of_sentences).toarray()[0].ravel()
    #print(np.mean(topic_vector))
    u['topic_tf_idf_vector'] = u.apply(lambda row: topic_vector, axis =1)
    new_df = pd.concat([new_df, u])

<ipython-input-342-8bce272cf7e3>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  u['tf_idf_vector'] = tf_idf_array
<ipython-input-342-8bce272cf7e3>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  u['topic_tf_idf_vector'] = u.apply(lambda row: topic_vector, axis =1)


In [344]:
additional_columns = vectorizer.get_feature_names_out()
print(len(names))
for name in names:
    print(name)
#for column in additional_columns:
    #print(column)
 #   test_df_task_ADU[column] = test_df_task_ADU.apply(lambda row: dictvalue_or_emphty(row['NER_analysis'], column), axis=1)

3981
ab
about
academy
acb
acclaim
action
activision
agence
al
alle
alphen
altman
america
amici
amicus
and
anderson
andreas
angel
ao
ap
apogee
arts
ass
association
atari
auto
avenging
bad
barbarian
bbfc
beam
big
blizzard
bloack
block
blood
blu
bmw
brain
breakers
bros
brown
bulletproof
bushman
call
can
capcom
carmageddon
carpocalypse
catalyst
cbs
cd
cent
centiorari
cero
certiorari
childhood
chosen
city
cloud
co
coffee
common
computer
comstock
conker
corporation
counter
crain
dance
day
dead
dealers
death
def
dejus
deselms
dill
dj
do
doom
dreamweb
ds
duke
duty
dvd
eagle
ec
eca
eidos
electronic
ema
entertainment
era
esa
esrb
et
everquest
evil
exidy
fallout
fantasy
fcc
ferguson
fight
final
fit
for
ford
forum
france
freedman
fur
gam
game
gamecube
games
gaming
gauntlett
gear
gears
geen
genesis
gentile
gore
grand
gt
gta
hac
halo
harpercollins
hartley
hawk
hedgehog
hot
hr
hurricane
id
idsa
ii
iii
instagib
interactive
interplay
invaders
iv
jam
journal
kaufhof
kilburn
kombat
ks
lan
left
legend
leh

лицензировать
лицензия
лицо
лично
личность
личный
лишение
лишенный
лобби
лоббистский
ловушка
логистика
ложный
локализация
локальноить
лонг
лонгитюдный
лоуренс
луи
луизиана
луизианы
луис
лучшея
лучшие
любимои
любимый
любой
людея
лёгкий
лёгкость
магазин
магнат
маиами
маикл
май
макет
маккомбс
маклюэн
максимальной
максимальный
малеинуть
маленький
мало
малое
малый
мальдонадо
мальчик
мандат
мандёр
манера
манипулировать
марио
маркер
маркетинг
маркетинговый
маркетолог
маркировать
маркироваться
маркировка
маркировкой
март
мартин
маршалл
масса
массачусетской
массив
массовой
массовый
мастер
масштаб
масштабный
материал
материальный
мать
машина
медиа
медиакультура
медиапродукция
медиин
медик
медицина
медицинский
медицинския
медицинской
медленно
медсестроить
между
межнациональный
межштатной
межэтнический
меинфреим
менее
меняться
меняющиися
мера
мероприятие
местной
место
месть
месяц
месячный
мета
метаанализ
метанарратив
метод
методологией
методология
механика
мешать
миллер
миллиард
миллион
минимальны

прошедшее
проявление
проявлять
прыгнуть
прямо
прямои
прямой
психиатр
психиатрический
психиатрия
психически
психический
психолог
психологически
психологический
психологическия
психологической
психология
публика
публикацией
публикация
публиковать
публиковаться
публично
публичный
пугать
пуля
пункт
пупок
пусть
путать
путь
путём
пуэртый
пытать
пытаться
пытка
пьеса
пять
рабадан
работа
работать
работоспособный
равенство
равно
равновесие
равный
ради
радио
раз
разбирательство
развивать
развивающеися
развивающийся
развитие
развлекательноить
развлекательный
развлечение
разгар
разгореться
раздел
разделимый
разделительный
разделить
разделять
различаться
различие
различноить
различный
размер
размещать
размещаться
размытие
размытый
размышление
разновидность
разногласие
разнообразный
разный
разоблачение
разовый
разозлить
разорвавшие
разочарование
разочаровывать
разработанноить
разработать
разработка
разработчик
разрешать
разрешаться
разрешить
разрушение
разрушить
разрыв
разумный
разъяснить
раион
рак
р

характер
характеристика
характерный
харрис
хватать
хватить
херб
хизер
хилла
хиллари
хирург
хит
ход
ходатаиство
холл
хороший
хорошо
хосе
хотеть
хотя
хранение
храниться
христианский
христианския
хуй
хьюсманна
цаоюань
целевой
целое
целый
цель
цензура
ценностея
ценность
ценообразование
центр
центральноить
цикл
цикличный
цикличёска
цитироваться
цифровой
цю
чапеть
час
частея
частично
частичный
частность
частный
часто
частый
часть
чей
человек
человеческий
человечество
чем
через
черта
чертой
четверо
четвертый
четверть
четкой
четыре
четырнадцатой
чжу
чинно
чиновник
число
чисто
чистый
читать
член
членство
чрезвычаин
чрезмерно
чрезмерный
чтение
что
чтобы
чувство
чувствовать
чуть
чэнвэя
чёрный
чёткий
чётко
шапиро
шахтёр
шварценеггер
шерил
шерри
шестнадцать
шестой
шесть
шефф
шизель
шиллер
широкий
широкия
широко
широкой
шкаф
школа
школьник
школьный
шлафля
шлёпать
шок
шокирующей
шон
шоу
шрам
штат
штраф
шум
шумовой
шутер
шутеры
щипать
эван
эверетта
эггерланда
экономический
экономическия
экономической


In [280]:
test_df_task_ADU = new_df

In [281]:
test_df_task_ADU['mean_tf_idf'] = test_df_task_ADU.apply(lambda row: np.mean(row['tf_idf_vector']), axis=1)

In [282]:
test_df_task_ADU['Topic_Sentence_Similarity_td_idf'] = test_df_task_ADU.apply(lambda row: cosine_similarity(row['topic_tf_idf_vector'], row['tf_idf_vector']), axis=1)

In [ ]:
for column in additional_columns:
    #print(column)
    test_df_task_ADU[column] = test_df_task_ADU.apply(lambda row: dictvalue_or_emphty(row['NER_analysis'], column), axis=1)

## Discource indicators

## Machine Learning

In [284]:
ML_tests = test_df_task_ADU.copy()

In [325]:
ML_tests.columns

Index(['ADU', 'End', 'Sentence_number', 'Sentence_position', 'Start',
       'string_length', 'amount_of_words', 'Topic_Sentence_Similarity', 'ORG',
       'LOC', 'PER', 'NOUN', 'ADP', 'ADV', 'VERB', 'DET', 'PUNCT', 'SCONJ',
       'CCONJ', 'ADJ', 'PROPN', 'AUX', 'NUM', 'PART', 'PRON', 'SYM', 'X',
       'INTJ', 'nsubj:pass', 'case', 'advmod', 'det', 'obl', 'punct', 'nmod',
       'conj', 'cc', 'amod', 'aux:pass', 'nsubj', 'parataxis', 'nummod', 'cop',
       'mark', 'ccomp', 'fixed', 'obj', 'appos', 'acl', 'xcomp', 'acl:relcl',
       'csubj', 'advcl', 'iobj', 'nummod:gov', 'discourse', 'flat:foreign',
       'flat:name', 'obl:agent', 'aux', 'flat', 'expl', 'orphan', 'compound',
       'csubj:pass', 'nummod:entity', 'list', 'mean_tf_idf',
       'Topic_Sentence_Similarity_td_idf'],
      dtype='object')

In [285]:
#Создаем функцию для удаления столбцов
def delete_columns(parameters, X):
    for i in parameters:
        try:
            del X[i]
        except Exception: 
            pass

In [286]:
parameters = ['Article_id',  'Sentence', 
       'Topic', 'Type', 'Russian_sentence','bag_of_words',
       'sentence_embedding', 'Russian_topic', 'Topic_embedding',
        'NER_analysis', 'NER_morph_analysis',
       'NER_syntax_analysis',  'array_of_words', 'tf_idf_vector','topic_tf_idf_vector']

In [287]:
delete_columns(parameters, ML_tests)

In [289]:
ML_tests

,ADU,End,Sentence_number,Sentence_position,Start,string_length,amount_of_words,Topic_Sentence_Similarity,ORG,LOC,...,aux,flat,expl,orphan,compound,csubj:pass,nummod:entity,list,mean_tf_idf,Topic_Sentence_Similarity_td_idf
2282,0.0,119.0,2.0,0.017699,3.0,165,18,0.540896,0,0,...,0,0,1,0,0,0,0,0,0.001744,0.000000
2283,0.0,311.0,3.0,0.026549,121.0,189,20,0.485801,0,0,...,0,0,0,0,0,0,0,0,0.002024,0.000000
2284,1.0,504.0,4.0,0.035398,313.0,203,21,0.301566,0,0,...,0,0,0,0,0,0,0,0,0.001961,0.000000
2285,0.0,750.0,5.0,0.044248,506.0,288,30,0.552495,1,0,...,0,0,0,0,0,0,0,0,0.002196,0.000000
2286,1.0,893.0,6.0,0.053097,752.0,170,18,0.439230,0,0,...,0,0,0,0,0,0,0,0,0.001754,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,0.0,1429.0,8.0,0.026578,1303.0,149,15,0.547210,0,0,...,0,0,0,0,0,0,0,0,0.000898,0.028772
961,0.0,1952.0,9.0,0.029900,1431.0,612,60,0.611786,1,0,...,0,0,0,0,0,0,0,0,0.001774,0.041263
962,0.0,2534.0,10.0,0.033223,1955.0,573,63,0.642910,0,0,...,0,0,0,0,0,0,0,0,0.001583,0.000000
963,0.0,2665.0,11.0,0.036545,2536.0,121,12,0.558885,0,0,...,0,0,0,0,0,0,0,0,0.000846,0.000000


In [318]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import cross_val_score, train_test_split
RANDOM_STATE = 17
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [311]:
#Разделяем данные на целевую переменную и 
ML_tests
ML_tests = ML_tests.dropna(axis='index', how='any')
y = ML_tests['ADU']
x = ML_tests.copy()
del x['ADU']

In [312]:
x.columns

Index(['End', 'Sentence_number', 'Sentence_position', 'Start', 'string_length',
       'amount_of_words', 'Topic_Sentence_Similarity', 'ORG', 'LOC', 'PER',
       'NOUN', 'ADP', 'ADV', 'VERB', 'DET', 'PUNCT', 'SCONJ', 'CCONJ', 'ADJ',
       'PROPN', 'AUX', 'NUM', 'PART', 'PRON', 'SYM', 'X', 'INTJ', 'nsubj:pass',
       'case', 'advmod', 'det', 'obl', 'punct', 'nmod', 'conj', 'cc', 'amod',
       'aux:pass', 'nsubj', 'parataxis', 'nummod', 'cop', 'mark', 'ccomp',
       'fixed', 'obj', 'appos', 'acl', 'xcomp', 'acl:relcl', 'csubj', 'advcl',
       'iobj', 'nummod:gov', 'discourse', 'flat:foreign', 'flat:name',
       'obl:agent', 'aux', 'flat', 'expl', 'orphan', 'compound', 'csubj:pass',
       'nummod:entity', 'list', 'mean_tf_idf',
       'Topic_Sentence_Similarity_td_idf'],
      dtype='object')

In [313]:
#Мы разделяем данные на тренировочную и тетобую выборку 70/30%
X_train, X_test, y_train, y_test = train_test_split(x, y,
                                                     test_size=0.3,
                                                     random_state=RANDOM_STATE)

In [314]:
#Просматриваем все столбцы по одной строке
for i in x:
    print(i, "\n",x[i][0])

End 
 248.0
Sentence_number 
 2.0
Sentence_position 
 0.007434944237918215
Start 
 4.0
string_length 
 260
amount_of_words 
 27
Topic_Sentence_Similarity 
 0.5858268872310477
ORG 
 0
LOC 
 0
PER 
 0
NOUN 
 14
ADP 
 3
ADV 
 1
VERB 
 1
DET 
 1
PUNCT 
 7
SCONJ 
 1
CCONJ 
 5
ADJ 
 8
PROPN 
 0
AUX 
 0
NUM 
 0
PART 
 0
PRON 
 0
SYM 
 0
X 
 0
INTJ 
 0
nsubj:pass 
 1
case 
 4
advmod 
 1
det 
 1
obl 
 2
punct 
 7
nmod 
 3
conj 
 10
cc 
 5
amod 
 5
aux:pass 
 0
nsubj 
 0
parataxis 
 0
nummod 
 0
cop 
 0
mark 
 0
ccomp 
 0
fixed 
 0
obj 
 0
appos 
 0
acl 
 0
xcomp 
 0
acl:relcl 
 0
csubj 
 0
advcl 
 0
iobj 
 0
nummod:gov 
 0
discourse 
 0
flat:foreign 
 0
flat:name 
 0
obl:agent 
 0
aux 
 0
flat 
 0
expl 
 0
orphan 
 0
compound 
 0
csubj:pass 
 0
nummod:entity 
 0
list 
 0
mean_tf_idf 
 0.0012077537524171921
Topic_Sentence_Similarity_td_idf 
 0.0


In [315]:
logisticRegr = LogisticRegression(random_state=0).fit(X_train, y_train)
logisticRegr.score(X_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8472382048331415

In [316]:
predictions = logisticRegr.predict(X_test)
score = logisticRegr.score(X_test, y_test)
print(score)

0.8402684563758389


In [322]:
print(f1_score(y_test, logisticRegr.predict(X_test)))
print(precision_score(y_test, logisticRegr.predict(X_test)))
print(recall_score(y_test, logisticRegr.predict(X_test)))

0.0916030534351145
0.3870967741935484
0.05194805194805195


In [ ]:
lm = LinearRegression().fit(X_train, y_train)
#Выводим коэффициенты, lm.intercept_ это w0
print(lm.coef_, lm.intercept_)
#Тут коэффициент детерминации считается, чем он больше, тем лучше
lm.score(X_train, y_train)

In [ ]:
SVM = svm.SVC()
clf.fit(X_train, y_train)
clf.predict([[2., 2.]])

In [ ]:
# from sklearn.svm import LinearSVC
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.datasets import make_classification
# X, y = make_classification(n_features=4, random_state=0)
# clf = make_pipeline(StandardScaler(),
# ...                     LinearSVC(random_state=0, tol=1e-5))
# >>> clf.fit(X, y)
# Pipeline(steps=[('standardscaler', StandardScaler()),
#                 ('linearsvc', LinearSVC(random_state=0, tol=1e-05))])

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.fit_transform(X_valid)

Что нужно сделать:
Будем делать как в Show Me Your Evidence – an Automatic Method for Context Dependent Evidence Detection
Для этого нужны:
тексты, темы, высказывания и доказательства
Однако есть датасеты, где есть высказывание, а есть за и против, возможно, мы можем противопостовление высказыванию выдать и сделать за то, что было против - это вот как раз в русском датасете, там просто надо было классифицировать на за и против
+Надо посмотреть, чтобы все мои данные соответствовали этой концепции
m-arg_multimodal-argumentation-dataset вот этот проверить
+Надо посмотреть два метода (но эти методы по идее для определения того, что на уровне ниже предложений)
Там в некоторых датасетах типа не предложения, а как раз аргументативные единицы, меньше предложения, поэтому надо понять, насколько это трудозатратно делать ,если делать
Eger et al. [8] proposed to use two cross-lingual techniques – direct transfer [15]
and projection [31] – for cross-lingual token-level argumentation mining

Дальше всё к одному виду привести
Дальше переводом все предложения на русский сделать
Дальше выписать все фичи, которые нужно в предложении выделить
Понять, как эти фичи выделить и выделить их
Дальше моделек несколько применить, описанных в работе
Дальше сделать евалюешн
Написать результаты

In [859]:
test1 = pd.read_csv('df_ADU_class.csv',sep=';', engine='python', header=None)
test1

,0,1,2,3
0,Topic,Sentence,ADU,Russian_sentence
1,This house believes that the sale of violent v...,A 2001 study found that exposure to violent vi...,True,"Исследование 2001 года показало, что воздейств..."
2,This house believes that the sale of violent v...,The most recent large scale meta-anlysis-- exa...,True,"Самый последний крупномасштабный мета-анализ, ..."
3,This house believes that the sale of violent v...,One study did find an increase in reports of b...,True,Одно исследование действительно обнаружило уве...
4,This house believes that the sale of violent v...,when one combines all relevant empirical studi...,True,если объединить все соответствующие эмпирическ...
...,...,...,...,...
6064,This house believes social deprivation causes ...,an environment of great inequalities between p...,True,среда большого неравенства между людьми может ...
6065,This house would enforce term limits on the le...,"The longer politicians are in office, the more...",True,"Чем дольше политики находятся у власти, тем бо..."
6066,This house would enforce term limits on the le...,The lack of mandatory limits to tenure is rega...,True,Отсутствие обязательных ограничений на пребыва...
6067,This house would enforce term limits on the le...,contact with the affairs of state is one of th...,True,контакт с государственными делами — одно из са...


In [ ]:
### https://github.com/UKPLab/acl2016-convincing-arguments

In [ ]:
X.groupby(['Day_bed'])['numeric_price'].count()
del X['']

In [ ]:
#Создаем функцию для удаления столбцов
def delete_columns(parameters, X):
    for i in parameters:
        try:
            del X[i]
        except Exception: 
            pass

In [ ]:
delete_columns(parameters, X)